In [1]:
# computational libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import scipy.stats as stats 
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy.interpolate import interp1d, RegularGridInterpolator, griddata, LinearNDInterpolator, NearestNDInterpolator
from math import ceil
from scipy import linalg
# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [2]:
lm = LinearRegression()
scale = StandardScaler()
qscale = QuantileTransformer()

# Project 3: Gradient Boost
## Implementing the Gradient Boosting Algorithm
First, we need our kernels, utility functions, and models that we have been using:

Kernels

In [3]:
# these kernels have been modified so that they can also handle weights matrices
# by default they do not handle weights matrices

# Gaussian Kernel
def Gaussian(x, wm = False):
  if not wm:
    if len(x.shape)==1:
      d = np.abs(x)
    else:
      d = np.sqrt(np.sum(x**2,axis=1))
    return np.where(d>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*d**2))
  else:
    return np.exp(-1/2*x**2)

# Tricubic Kernel
def Tricubic(x, wm = False):
  if not wm:
    if len(x.shape) == 1:
      x = x.reshape(-1,1)
    d = np.sqrt(np.sum(x**2,axis=1))
    return np.where(d>1,0,70/81*(1-d**3)**3)
  else:
    return (1 - x ** 3) ** 3

# Quartic Kernel
def Quartic(x, wm = False):
  if not wm:
    if len(x.shape) == 1:
      x = x.reshape(-1,1)
    d = np.sqrt(np.sum(x**2,axis=1))
    return np.where(d>1,0,15/16*(1-d**2)**2)
  else:
    return (1 - x ** 2) ** 2

# Epanechnikov Kernel
def Epanechnikov(x, wm = False):
  if not wm:
    if len(x.shape) == 1:
      x = x.reshape(-1,1)
    d = np.sqrt(np.sum(x**2,axis=1))
    return np.where(d>1,0,3/4*(1-d**2)) 
  else:
    return (1 - x ** 2)

Utility Functions

In [4]:
def kernel_function(xi,x0,kern, tau): 
  return kern((xi - x0)/(2*tau))

def weights_matrix(x,x_new,kern,tau):
  if np.isscalar(x_new):
    return kernel_function(x,x_new,kern,tau)
  else:
    n = len(x_new)
    return np.array([kernel_function(x,x_new[i],kern,tau) for i in range(n)])

def dist(u,v):
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

Models

In [5]:
class Lowess:
  def __init__(self, kernel = Tricubic, tau=0.05):
    self.kernel = kernel
    self.tau = tau
  
  def fit(self, x, y):
    kernel = self.kernel
    tau = self.tau
    self.xtrain_ = x
    self.yhat_ = y

  def predict(self, x_new):
    check_is_fitted(self)
    x = self.xtrain_
    y = self.yhat_

    w = weights_matrix(x,x_new,self.kernel,self.tau)

    if np.isscalar(x_new):
      lm.fit(np.diag(w).dot(x.reshape(-1,1)),np.diag(w).dot(y.reshape(-1,1)))
      yest = lm.predict([[x_new]])[0][0]
    elif len(x.shape)==1:
      n = len(x_new)
      yest_test = np.zeros(n)
      #Looping through all x-points
      for i in range(n):
        lm.fit(np.diag(w[i,:]).dot(x.reshape(-1,1)),np.diag(w[i,:]).dot(y.reshape(-1,1)))
        yest_test[i] = lm.predict(x_new[i].reshape(-1,1))
    else:
      n = len(x_new)
      yest_test = np.zeros(n)
      #Looping through all x-points
      for i in range(n):
        lm.fit(np.diag(w[i,:]).dot(x),np.diag(w[i,:]).dot(y.reshape(-1,1)))
        yest_test[i] = lm.predict(x_new[i].reshape(1,-1))
    return yest_test

  def get_params(self, deep=True):
    return {"kernel":self.kernel, "tau":self.tau}

  def set_params(self, **parameters):
    for parameter, value in parameters.items():
        setattr(self, parameter, value)
    return self

def lowess_ag_md(x, y, xnew, f=2/3, iter=3, intercept=True, kernel=Tricubic):

  n = len(x)

  # calculating the amount of neighbors we are considering
  r = int(ceil(f * n))

  # one dimensional y and x become column vectors here
  if len(y.shape)==1:
    y = y.reshape(-1,1)

  if len(x.shape)==1:
    x = x.reshape(-1,1)

  # we need to add a column of ones to account for the intercept
  # this is by default, however the user has the option to not 
  # include this step
  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x

  # calculating the Euclidean distance for each observation
  h = [np.sort(np.sqrt(np.sum((x-x[i])**2,axis=1)))[r] for i in range(n)]

  # we now clip any distance that falls outside of our bounds
  w = np.clip(dist(x,x) / h, 0.0, 1.0)

  # we apply our kernel to get the weights, giving closer neighbors more weight
  # by default, we use tricubic, however, the user may specify an alternate
  w = kernel(w, True)

  # initialize our y estimate and delta
  yest = np.zeros(n)
  delta = np.ones(n)

  for iteration in range(iter):
    for i in range(n):
      # convert our weights to a diagonal matrix
      W = np.diag(delta).dot(np.diag(w[i,:]))

      # this is our ordinate values
      b = np.transpose(x1).dot(W).dot(y)

      # this is our coefficient matrix
      A = np.transpose(x1).dot(W).dot(x1)

      # L2 regularization, this ensures that our matrix equation can always be solved
      A = A + 0.0001*np.eye(x1.shape[1])

      # finally, we solve our matrix equation
      beta = linalg.solve(A, b)

      # create the y estimate
      yest[i] = np.dot(x1[i],beta)

    # now we get the residuals and calculate our cutoff for outliers
    residuals = y - yest

    # quartiles and IQR
    Q1 = np.percentile(residuals,25)
    Q3 = np.percentile(residuals,75)
    IQR = Q3-Q1

    # upper and lower bound for outliers
    upper = Q3 + (1.5 * IQR)
    lower = Q1 = (1.5 * IQR)

    # these values help us standardize the data to have the lower bound be -1
    # and the upper bound be 1
    mid = (upper + lower) / 2
    div = upper-mid

    # finally, we clip all outliers
    delta = np.clip((residuals - mid)/div, -1, 1)
    delta = (1 - delta ** 2) ** 2

  # this is how we handle training and testing data (x, xnew)
  if x.shape[1]==1:
    # this is a one dimensional interpolator
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
    output = f(xnew)
  else:
    # otherwise, we need to use principal component analysis on our higher dimensional data
    # this is because there may be relationships between some of the features
    output = np.zeros(len(xnew))
    for i in range(len(xnew)):
      ind = np.argsort(np.sqrt(np.sum((x-xnew[i])**2,axis=1)))[:r]
      pca = PCA(n_components=3)
      x_pca = pca.fit_transform(x[ind])
      tri = Delaunay(x_pca,qhull_options='QJ')
      f = LinearNDInterpolator(tri,y[ind])
      output[i] = f(pca.transform(xnew[i].reshape(1,-1)))

  # the output may have NaN's where the data points from xnew are outside the convex hull of X
  # here, we will resolve that with nearest neighbor interpolation in higher dimensions
  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,y.ravel()) 
    output[np.isnan(output)] = g(xnew[np.isnan(output)])
  return output

class Lowess_AG_MD:
  def __init__(self, f = 1/10, iter = 3,intercept=True, kernel=Tricubic):
    self.f = f
    self.iter = iter
    self.intercept = intercept
    self.kernel = kernel
  
  def fit(self, x, y):
    f = self.f
    iter = self.iter
    self.xtrain_ = x
    self.yhat_ = y

  def predict(self, x_new):
    check_is_fitted(self)
    x = self.xtrain_
    y = self.yhat_
    f = self.f
    iter = self.iter
    intercept = self.intercept
    kernel = self.kernel
    return lowess_ag_md(x, y, x_new, f, iter, intercept, kernel)

  def get_params(self, deep=True):
    return {"f": self.f, "iter": self.iter,"intercept":self.intercept,"kernel":self.kernel}

  def set_params(self, **parameters):
    for parameter, value in parameters.items():
        setattr(self, parameter, value)
    return self

Now that we have our kernels, distance function, and SciKit-Learn Compliant Multidimensional Lowess Model, we are ready to implement our Gradient Boosting Algorithm.

In [6]:
def boosted_lwr(x, y, xnew, model1=Lowess_AG_MD, model2=Lowess_AG_MD, **kwargs):
  model1 = model1(**kwargs)
  model1.fit(x,y)
  residuals1 = y - model1.predict(x)
  model2 = model2(**kwargs)
  model2.fit(x,residuals1)
  output = model1.predict(xnew) + model2.predict(xnew)
  return output 

## Testing Regressor Across Different Datasets and Kernels


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Cars Dataset

In [8]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/cars.csv')
data.head(5)

,MPG,CYL,ENG,WGT
0,18.0,8,307.0,3504
1,15.0,8,350.0,3693
2,18.0,8,318.0,3436
3,16.0,8,304.0,3433
4,17.0,8,302.0,3449


In [88]:
x = data.loc[:,'CYL':'WGT'].values
y = data['MPG'].values
xtrain, xtest, ytrain, ytest = tts(x,y,test_size=0.3,shuffle=True,random_state=123)

Tricubic Kernel


In [25]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.3,iter=1,intercept=True,kernel=Tricubic)

In [22]:
mse(ytest,yhat)

22.625729222794533

Epanechnikov Kernel

In [26]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.3,iter=1,intercept=True,kernel=Epanechnikov)

In [27]:
mse(ytest,yhat)

22.625729222794533

Gaussian Kernel

In [30]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.3,iter=1,intercept=True,kernel=Gaussian)

In [31]:
mse(ytest,yhat)

22.625729222794533

Quartic Kernel

In [32]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.3,iter=1,intercept=True,kernel=Quartic)

In [33]:
mse(ytest,yhat)

22.625729222794533

### Concrete Dataset

In [34]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/concrete.csv')
data.head(5)

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [35]:
x = data.loc[:,'cement':'age'].values
y = data['strength'].values
xtrain, xtest, ytrain, ytest = tts(x,y,test_size=0.3,shuffle=True,random_state=123)

Tricubic Kernel


In [50]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.015,iter=1,intercept=True,kernel=Tricubic)

In [51]:
mse(ytest,yhat)

54.01967739424208

Epanechnikov Kernel

In [32]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.015,iter=1,intercept=True,kernel=Epanechnikov)

In [52]:
mse(ytest,yhat)

54.01967739424208

Gaussian Kernel

In [34]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.015,iter=1,intercept=True,kernel=Gaussian)

In [53]:
mse(ytest,yhat)

54.01967739424208

Quartic Kernel

In [36]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.015,iter=1,intercept=True,kernel=Quartic)

In [54]:
mse(ytest,yhat)

54.01967739424208

### Housing Dataset

In [55]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/housing.csv')
data.head(5)

,town,tract,longitude,latitude,crime,residential,industrial,river,nox,rooms,older,distance,highway,tax,ptratio,lstat,cmedv
0,Nahant,2011,-70.955002,42.255001,0.00632,18.0,2.31,no,0.538,6.575,65.199997,4.0900,1,296,15.300000,4.98,24.000000
1,Swampscott,2021,-70.949997,42.287498,0.02731,0.0,7.07,no,0.469,6.421,78.900002,4.9671,2,242,17.799999,9.14,21.600000
2,Swampscott,2022,-70.935997,42.283001,0.02729,0.0,7.07,no,0.469,7.185,61.099998,4.9671,2,242,17.799999,4.03,34.700001
3,Marblehead,2031,-70.928001,42.292999,0.03237,0.0,2.18,no,0.458,6.998,45.799999,6.0622,3,222,18.700001,2.94,33.400002
4,Marblehead,2032,-70.921997,42.298000,0.06905,0.0,2.18,no,0.458,7.147,54.200001,6.0622,3,222,18.700001,5.33,36.200001


In [121]:
features = ['crime','rooms','residential','industrial','nox','older','distance','highway','tax','ptratio','lstat']
x = np.array(data[features])
y = data['cmedv'].values
xtrain, xtest, ytrain, ytest = tts(x,y,test_size=0.3,shuffle=True,random_state=123)

Tricubic Kernel


In [85]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.3,iter=1,intercept=True,kernel=Tricubic)

In [126]:
mse(ytest,yhat)

29.054248167685376

Epanechnikov Kernel

In [74]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.1,iter=1,intercept=True,kernel=Epanechnikov)

In [75]:
mse(ytest,yhat)

29.054248167685376

Gaussian Kernel

In [124]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.1,iter=1,intercept=True,kernel=Gaussian)

In [125]:
mse(ytest,yhat)

29.054248167685376

Quartic Kernel

In [78]:
yhat = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.1,iter=1,intercept=True,kernel=Quartic)

In [79]:
mse(ytest,yhat)

29.054248167685376

As we can see, we got the same mse for each of the kernels for the three datasets that we used. Because of this, we will select Tricubic to proceed with.

## K-Fold Cross-Validation
### Random Forest Comparison

In [98]:
# Cars
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/cars.csv')
x = data.loc[:,'CYL':'WGT'].values
y = data['MPG'].values

mse_lwr = []
mse_lwr_ag = []
mse_rf = []
mse_lwr_b = []
mse_lwr_ag_b = []
mse_rf_b = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_lw = Lowess(kernel=Gaussian,tau=0.18)
model_lw_ag = Lowess_AG_MD(f=1/3,iter=1,intercept=True,kernel=Tricubic)
model_rf = RandomForestRegressor(n_estimators=200,max_depth=3)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  # Lowess
  model_lw.fit(xtrain,ytrain)
  yhat_lw = model_lw.predict(xtest)

  yhat_lw_b = boosted_lwr(xtrain,ytrain,xtest,Lowess,Lowess,kernel=Gaussian,tau=0.18)

  # Lowess Gramfort
  model_lw_ag.fit(xtrain,ytrain)
  yhat_lw_ag = model_lw_ag.predict(xtest)
  
  yhat_lw_ag_b = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=1/3,iter=1,intercept=True,kernel=Tricubic)
  
  # Random Forest
  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  yhat_rf_b = boosted_lwr(xtrain,ytrain,xtest,RandomForestRegressor,RandomForestRegressor,n_estimators=200,max_depth=3)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_lwr_b.append(mse(ytest,yhat_lw_b))
  mse_lwr_ag.append(mse(ytest,yhat_lw_ag))
  mse_lwr_ag_b.append(mse(ytest,yhat_lw_ag_b))
  mse_rf.append(mse(ytest,yhat_rf))
  mse_rf_b.append(mse(ytest,yhat_rf_b))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for Boosted Locally Weighted Regression is : '+str(np.mean(mse_lwr_b)))
print("The Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : "+str(np.mean(mse_lwr_ag)))
print("The Cross-validated Mean Squared Error for Boosted Gramfort's Locally Weighted Regression is : "+str(np.mean(mse_lwr_ag_b)))
print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))
print('The Cross-validated Mean Squared Error for Boosted Random Forest is : '+str(np.mean(mse_rf_b)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 17.133192203301153
The Cross-validated Mean Squared Error for Boosted Locally Weighted Regression is : 17.384268463244858
The Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : 23.53685564215548
The Cross-validated Mean Squared Error for Boosted Gramfort's Locally Weighted Regression is : 23.53357188789293
The Cross-validated Mean Squared Error for Random Forest is : 16.621845168912557
The Cross-validated Mean Squared Error for Boosted Random Forest is : 16.20216483653494


In [114]:
# Concrete
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/concrete.csv')
x = data.loc[:,'cement':'age'].values
y = data['strength'].values

mse_lwr = []
mse_lwr_ag = []
mse_rf = []
mse_lwr_b = []
mse_lwr_ag_b = []
mse_rf_b = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_lw = Lowess(kernel=Gaussian,tau=0.04)
model_lw_ag = Lowess_AG_MD(f=0.015,iter=1,intercept=True,kernel=Tricubic)
model_rf = RandomForestRegressor(n_estimators=200,max_depth=3)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  # Lowess
  model_lw.fit(xtrain,ytrain)
  yhat_lw = model_lw.predict(xtest)

  yhat_lw_b = boosted_lwr(xtrain,ytrain,xtest,Lowess,Lowess,kernel=Gaussian,tau=0.04)

  # Lowess Gramfort
  model_lw_ag.fit(xtrain,ytrain)
  yhat_lw_ag = model_lw_ag.predict(xtest)
  
  yhat_lw_ag_b = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.015,iter=1,intercept=True,kernel=Tricubic)
  
  # Random Forest
  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  yhat_rf_b = boosted_lwr(xtrain,ytrain,xtest,RandomForestRegressor,RandomForestRegressor,n_estimators=200,max_depth=3)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_lwr_b.append(mse(ytest,yhat_lw_b))
  mse_lwr_ag.append(mse(ytest,yhat_lw_ag))
  mse_lwr_ag_b.append(mse(ytest,yhat_lw_ag_b))
  mse_rf.append(mse(ytest,yhat_rf))
  mse_rf_b.append(mse(ytest,yhat_rf_b))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for Boosted Locally Weighted Regression is : '+str(np.mean(mse_lwr_b)))
print("The Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : "+str(np.mean(mse_lwr_ag)))
print("The Cross-validated Mean Squared Error for Boosted Gramfort's Locally Weighted Regression is : "+str(np.mean(mse_lwr_ag_b)))
print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))
print('The Cross-validated Mean Squared Error for Boosted Random Forest is : '+str(np.mean(mse_rf_b)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 800.4766226078477
The Cross-validated Mean Squared Error for Boosted Locally Weighted Regression is : 800.4154255037517
The Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : 44.72821371004521
The Cross-validated Mean Squared Error for Boosted Gramfort's Locally Weighted Regression is : 44.88046414089581
The Cross-validated Mean Squared Error for Random Forest is : 86.64659092779175
The Cross-validated Mean Squared Error for Boosted Random Forest is : 55.39960750145254


In [130]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/data/housing.csv')
features = ['crime','rooms','residential','industrial','nox','older','distance','highway','tax','ptratio','lstat']
x = np.array(data[features])
y = data['cmedv'].values

mse_lwr = []
mse_lwr_ag = []
mse_rf = []
mse_lwr_b = []
mse_lwr_ag_b = []
mse_rf_b = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_lw = Lowess(kernel=Gaussian,tau=0.17)
model_lw_ag = Lowess_AG_MD(f=0.1,iter=1,intercept=True,kernel=Tricubic)
model_rf = RandomForestRegressor(n_estimators=200,max_depth=3)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  # Lowess
  model_lw.fit(xtrain,ytrain)
  yhat_lw = model_lw.predict(xtest)

  yhat_lw_b = boosted_lwr(xtrain,ytrain,xtest,Lowess,Lowess,kernel=Gaussian,tau=0.17)

  # Lowess Gramfort
  model_lw_ag.fit(xtrain,ytrain)
  yhat_lw_ag = model_lw_ag.predict(xtest)
  
  yhat_lw_ag_b = boosted_lwr(xtrain,ytrain,xtest,Lowess_AG_MD,Lowess_AG_MD,f=0.1,iter=1,intercept=True,kernel=Tricubic)
  
  # Random Forest
  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  yhat_rf_b = boosted_lwr(xtrain,ytrain,xtest,RandomForestRegressor,RandomForestRegressor,n_estimators=200,max_depth=3)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_lwr_b.append(mse(ytest,yhat_lw_b))
  mse_lwr_ag.append(mse(ytest,yhat_lw_ag))
  mse_lwr_ag_b.append(mse(ytest,yhat_lw_ag_b))
  mse_rf.append(mse(ytest,yhat_rf))
  mse_rf_b.append(mse(ytest,yhat_rf_b))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for Boosted Locally Weighted Regression is : '+str(np.mean(mse_lwr_b)))
print("The Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : "+str(np.mean(mse_lwr_ag)))
print("The Cross-validated Mean Squared Error for Boosted Gramfort's Locally Weighted Regression is : "+str(np.mean(mse_lwr_ag_b)))
print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))
print('The Cross-validated Mean Squared Error for Boosted Random Forest is : '+str(np.mean(mse_rf_b)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 121.22597090893612
The Cross-validated Mean Squared Error for Boosted Locally Weighted Regression is : 121.92895063506433
The Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : 16.12087329403363
The Cross-validated Mean Squared Error for Boosted Gramfort's Locally Weighted Regression is : 16.12087329403365
The Cross-validated Mean Squared Error for Random Forest is : 17.665804853731764
The Cross-validated Mean Squared Error for Boosted Random Forest is : 13.852908916916466
